# Module 5 Homework - Let's Get Lunch

Suppose we are looking for a good place for lunch on our next trip to La Crosse. Let's collect some data on nearby restaurants.

## <font color="red"> Problem 1 </font>

Goto yelp.com and perform a search with the following parameters

* **Find** Restaurants
* **Near** La Crosse, WI

**Tasks**
1. Copy the resulting web address below and determine the how the     specified search terms related to the resulting address
2. Use requests and Beautiful Soap to download the content of the front page.

In [76]:
!pip install composable --upgrade
!pip install composablesoup --upgrade

Requirement already up-to-date: composable in /home/dcmdillon/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.5)
Requirement already up-to-date: composablesoup in /home/dcmdillon/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.4)


In [77]:
from composable import pipeable
from composable.strict import map, filter
from composablesoup import find, find_all, get_text, has_attr
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling
from composable.sequence import to_list, head
from composable.string import strip
from composable import from_toolz as tlz


In [78]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI')
food = BeautifulSoup(r.content, 'html.parser')

## <font color="red"> Problem 2 </font>

We want to grab the restaurant's name.

1. Use Inspect Element to determine the tags/classes for each of the elements listed above.  
2. Note that all the business information is contained in a`div` that contains a class similar to  `<div class=" ... businessName__09f24__3Wql2 ...">`.  You will need to use a regular expression to match the `businessName` in the class (see lecture 5.3).
3. Write expressions/functions to pull out the name of each restaurant.  
    * Note: The business name is in an unnamed tag, you will need to navigate to the using searches and/or relationship
 
**Confirm that there is an extra restaurant in the list (e.g. 11-12 instead of 10). This is due to an advertisement/sponsered links, which we want to ignore.**

In [79]:
import re
result = (food
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>map(find('a'))
 >>map(get_text)
)
result


['Pappi’s Taqueria y Mas',
 'The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'Piggy’s Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher']

In [80]:
import re
remove_ad = pipeable(lambda ls: [li for li in ls if re.match("\d",get_text(li))])
get_name = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >>map(find('a'))
 >>map(get_text)
 
)

In [81]:
get_name(food)

['The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'Piggy’s Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher']

## <font color="red"> Problem 3 </font>

Since we picked up extra information, we will need to be clever about identifying the information block for each restaurant.  Note that all of the actual search results (but not sponsered links) start with the ranking (e.g. `11.`).  Use the following steps to get a list that contains the information for each restaurant other than the adds.

1. Start by finding the ranking of the restaurant (1., 2., etc.). **Hint:** You will need to use regular expression to match the text of the tag (see lecture 5.3).
2. Now search for a parent of the above tags that surrounds all of the restaurant information.  You will want to use the `find_parent` method on each of the tags from **1.**.  **Hint:** Look through each of the `div` tags that contain the ranking, looking for a meaningful tag name to match with a regular expression.

The resulting list will be the starting point for gathering all of the information.

In [82]:
seperate_rank = pipeable(lambda ls: [li.split(".")[0] for li in ls])
result = (food
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >>map(find('span'))
 >>map(get_text)
 >>seperate_rank
)
result

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [83]:
get_rank = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >>map(find('span'))
 >>map(get_text)
 >>seperate_rank
)
get_rank(food)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

## <font color="red"> Problem 4 </font>

Write expressions/functions to gather each of the following pieces of information for each of the restaurants.

#### Restaurant Name

In [84]:
get_name = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >>map(find('a'))
 >>map(get_text)
 
)
get_name(food)

['The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'Piggy’s Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher']

#### Rating

In [85]:
get_rating = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >> map(find_parent)
 >> map(find_parent)
 >> map(find('div', attrs={'class': re.compile('i-stars')}))
 >> filter(has_attr('aria-label'))
 >> map(tlz.get('aria-label'))
)
get_rating(food)

['4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '4 star rating',
 '3.5 star rating',
 '4.5 star rating']

#### Address

In [86]:
get_address = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >>map(find_next_sibling)
 >>map(find('address'))
 >>map(get_text) 
)
get_address(food)

['328 Front St S',
 '1810 State St',
 '222 Pearl St',
 '300 3rd St S',
 '101 State St',
 '107 Vine St',
 '1128 La Crosse St',
 '501 Front St S',
 '100 3rd St S',
 '321 State St']

#### Review Count

In [87]:
get_reviewCount = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >> map(find_parent)
 >> map(find_parent)
 >> map(find('span', attrs={'class': re.compile('reviewCount')}))
 >>map(get_text)
)

get_reviewCount(food)

['226', '17', '275', '109', '149', '125', '57', '131', '154', '17']

#### Category

**Examples:** `['American (New)', 'Seafood', 'Steakhouses']` becomes `'American (New);Seafood;Steakhouses'`

In [88]:
remove_DollarSigns = pipeable(lambda l: l.replace("$",""))
replace_Commas = pipeable(lambda l: l.replace(", ",";"))

get_GetCategory = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >> map(find_parent)
 >> map(find_parent)
 >> map(find('div', attrs={'class': re.compile('priceCategory')}))
 >>map(get_text)
 >>map(remove_DollarSigns)
 >>map(replace_Commas)
)

In [89]:
get_GetCategory(food)

['American (New);Seafood;Steakhouses',
 'American (Traditional)',
 'American (Traditional);Cajun/Creole',
 'American (New)',
 'French;Cocktail Bars',
 'Seafood;Steakhouses;Desserts',
 'American (New);Pubs',
 'Steakhouses;Seafood;Sandwiches',
 'American (Traditional);Gastropubs',
 'Butcher;Delis;Caterers']

## <font color="red">  Problem 4 </font>

Package all of the expressions in a function that takes a url as input and returns the table of information.  Use a `def` statement and put the above helper functions in the body of the main function.  Test this function on the front page of the search.

In [100]:
remove_ad = pipeable(lambda ls: [li for li in ls if re.match("\d",get_text(li))])
seperate_rank = pipeable(lambda ls: [li.split(".")[0] for li in ls])
remove_DollarSigns = pipeable(lambda l: l.replace("$",""))
replace_Commas = pipeable(lambda l: l.replace(", ",";"))
safe_getText = pipeable(lambda l: l.text if l is not None else "")
safe_getStars = pipeable(lambda l: l["aria-label"] if l is not None else "")


get_name = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >>map(find('a'))
 >>map(safe_getText)
)

get_rank = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >>map(find('span'))
 >>map(safe_getText)
 >>seperate_rank
)

get_rating = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >> map(find_parent)
 >> map(find_parent)
 >> map(find('div', attrs={'class': re.compile('i-stars')}))
 >> map(safe_getStars)
)

get_address = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >>map(find_next_sibling)
 >>map(find('address'))
 >>map(safe_getText) 
)

get_reviewCount = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >> map(find_parent)
 >> map(find_parent)
 >> map(find('span', attrs={'class': re.compile('reviewCount')}))
 >>map(safe_getText)
)

get_GetCategory = pipeable(lambda soup: soup
 >> find_all('div', attrs={'class': re.compile('businessName')})
 >>remove_ad
 >> map(find_parent)
 >> map(find_parent)
 >> map(find('div', attrs={'class': re.compile('priceCategory')}))
 >>map(safe_getText)
 >>map(remove_DollarSigns)
 >>map(replace_Commas)
)

In [101]:
def yelpScrape(url):
    print("Proccesing Page {0}".format(url))
    s = requests.Session()
    r = s.get(url)
    food = BeautifulSoup(r.content, 'html.parser')
    
    data = [list(a) for a in zip(get_name(food),get_rank(food),get_rating(food),get_address(food),get_reviewCount(food),get_GetCategory(food))]
    return data


yelpScrape("https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=10")

Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=10


[['Digger’s Sting Restaurant',
  '11',
  '4 star rating',
  '122 3rd St N',
  '62',
  'American (New);Steakhouses'],
 ['4 Sisters Wine Bar and Tapas',
  '12',
  '3.5 star rating',
  '100 Harborview Plz',
  '111',
  'Wine Bars;Tapas/Small Plates;Tapas Bars'],
 ['River Rats Bar and Grill',
  '13',
  '4.5 star rating',
  '1311 La Crescent Pl',
  '28',
  'American (New);Burgers;Cocktail Bars'],
 ['Uno Venti Pizzeria',
  '14',
  '4 star rating',
  '120 King St',
  '17',
  'Pizza;Italian;Beer Bar'],
 ['Milwaukee Burger Company',
  '15',
  '3.5 star rating',
  '3039 Medco Ct',
  '23',
  'Burgers;Sports Bars;Beer Bar'],
 ['Le Chateau',
  '16',
  '4.5 star rating',
  '410 Cass St',
  '51',
  'French;Wine Bars;Cocktail Bars'],
 ['Burritos House',
  '17',
  '4.5 star rating',
  '1205 La Crosse St',
  '30',
  'Mexican'],
 ['Señor Villa', '18', '4 star rating', '325 West Ave N', '69', 'Mexican'],
 ['Red Pines Bar & Grill',
  '19',
  '4.5 star rating',
  'W7305 County Rd Z',
  '88',
  'American (Tra

## <font color="red">  Problem 5 </font>

Now perform a linear search to grab all of the information on restaurants in La Crosse.  You will need to

1. Inspect the url for the first, second, etc. pages to determine the pattern.
2. Create a list of urls for all search results.
3. Get the info from all pages and use your function from the last problem to parse the data into a list.
4. Write the results to a csv file. **Hint:** Use `'a'` (append) instead of `'w'` write on all after the first/

In [102]:
yelpLink = lambda n: f"https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start={n}"

yelpLink(10)

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=10'

In [103]:
urls = [yelpLink(i) for i in range(0,240,10)]

urls

['https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=0',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=10',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=20',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=30',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=40',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=50',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=60',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=70',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=80',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=90',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Cross

In [104]:
with open('yelp_data.csv','w') as outfile:
    outfile.write("Name,Rank,Rating,Address,Review Count,Category\n")
            
data = [yelpScrape(url) for url in urls ]


Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=0
Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=10
Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=20
Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=30
Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=40
Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=50
Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=60
Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=70
Proccesing Page https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&&start=80
Proccesing Page https://www.yelp.com/search?find_desc=Re

In [105]:
data

[[['The Waterfront Restaurant & Tavern',
   '1',
   '4.5 star rating',
   '328 Front St S',
   '226',
   'American (New);Seafood;Steakhouses'],
  ['Restore Public House',
   '2',
   '4.5 star rating',
   '1810 State St',
   '17',
   'American (Traditional)'],
  ['Buzzard Billy’s',
   '3',
   '4 star rating',
   '222 Pearl St',
   '275',
   'American (Traditional);Cajun/Creole'],
  ['Lovechild Restaurant',
   '4',
   '4.5 star rating',
   '300 3rd St S',
   '109',
   'American (New)'],
  ['The Charmant',
   '5',
   '4.5 star rating',
   '101 State St',
   '149',
   'French;Cocktail Bars'],
  ['The Freighthouse Restaurant',
   '6',
   '4 star rating',
   '107 Vine St',
   '125',
   'Seafood;Steakhouses;Desserts'],
  ['Howie’s on La Crosse',
   '7',
   '4 star rating',
   '1128 La Crosse St',
   '57',
   'American (New);Pubs'],
  ['Piggy’s Restaurant',
   '8',
   '4 star rating',
   '501 Front St S',
   '131',
   'Steakhouses;Seafood;Sandwiches'],
  ['The Crow',
   '9',
   '3.5 star ratin

In [106]:
lines = [",".join(r) for page in data for r in page]
content = "\n".join(lines)

In [107]:
with open('yelp_data.csv','a') as outfile:  
        outfile.write(content)

## <font color="red">  Bonus Problem </font>

See if you can also grab the latitude and longitude of each result.